In [1]:
import sys
import time
import random
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

from ipypb import irange
%load_ext autotime

In [32]:
credit_list = []


def one(search):    
    title = split(search)
    urlset = url_machine(imdb(title), title)
    return clean(google(urlset))


def split(search):
    target = ''
    target_list = search.split(' ')

    for i in range(len(target_list)):
        if i == len(target_list) - 1:
            target += target_list[i]
        else:
            target += target_list[i] + '+'
    
    return target
        
    
def clean(data):
    active = False

    #Ranking
    for i in range(0, len(data) - 1):
        cur = i

        for j in range(i + 1, len(data) - 1):
            if data[j][0] > data[cur][0]:
                cur = j
                active = True

        if (active):
            spare = data[i]
            data[i] = data[cur]
            data[cur] = spare
            active = False
            
    #Trimming
    for i in range(0, len(data) - 1):
        cur = i

        try:
            for j in range(i + 1, len(data) - 1):
                if data[j][1] == data[cur][1]:
                    data.pop(j)
        except:
            pass
    
    return data


def google(info):
    package = []
    
    for i in range(len(info)):
        seq_url = info[i][0]
        seq_weight = info[i][1]
        seq_page = requests.get(seq_url)
        seq_page.encoding = 'utf-8'
        soup = bs(seq_page.text, 'lxml')
        result = soup.find_all("div", class_="kCrYT")
        entry_rank = 0
        url = ''
        domain = ''
        sypnosis = ''

        for j in range(len(result)):
            try:
                url = result[j].select('a')[0].get('href').split('&')[0].split('=')[1]
                domain = url.split('/')[2]
            except:
                pass
            
            try:
                sypnosis = result[j].find_all('div', class_='BNeawe')[0].find_all('div', class_='BNeawe')[0].get_text()
            except:
                pass
            
            weight = round(seq_weight * (10 - entry_rank) / 10, 3)
            entry = [weight, domain, sypnosis, url]
            if j % 2 == 1:
                package.append(entry)
            else:
                entry_rank += 1
            
        package.append(entry)
        
    return package


def url_machine(info, title):
    booster = "+'film'or'movie'or'series'or'television'or'TV'"
    url_list = []
    
    for i in range(len(info)):
        credit = info[i][0]
        if credit == 'Director' or credit == 'Directors':
            credit_weight = 2
            weight_list = [1, 0.75, 0.65]
        elif credit == 'Writer' or credit == 'Writers':
            credit_weight = 1.75
            weight_list = [1, 0.85, 0.8]
        elif credit == 'Star' or credit == 'Stars':
            credit_weight = 1.66
            weight_list = [1, 0.9, 0.85]
        
        for j in range(len(info[i])):
            if j != 0:
                url = 'https://www.google.com/search?q=' + info[i][j] + '+' + title + booster
                url += "+-site:www.imdb.com" + "+-site:www.rottentomatoes.com" + "+-site:www.metacritic.com"
                url += "+-site:wikipedia.org" + "+-site:pinterest.com" + "+-site:www.amazon.com"
                url += "+-site:facebook.com" + "+-site:twitter.com" + "+-site:instagram.com"
                url += "+-site:youtube.com"
                url_list.append([url, credit_weight * weight_list[j - 1]])
    
    return url_list


# Attempting to utilize IMDb API

def imdb(title):
    api_title = title.replace("+", "%20")
    imdb_api0 = requests.get("https://imdb-api.com/en/API/Search/k_g3y2mnmy/" + api_title)
    imdb_result = bs(imdb_api0.text, 'lxml')
    imdb_id = json.loads(imdb_result.select('p')[0].get_text())['results'][0]['id']
    imdb_api1 = requests.get("https://imdb-api.com/en/API/FullCast/k_g3y2mnmy/" + str(imdb_id))
    imdb_table = json.loads(bs(imdb_api1.text, 'lxml').select('p')[0].get_text())
    
    director = imdb_table['directors']['items'][0]['name'] # director
    writer = imdb_table['writers']['items'][0]['name'] # writer
    actor0 = imdb_table['actors'][1]['name'] # actor
    actor1 = imdb_table['actors'][2]['name'] # actor
    
    credit_list = [['Director', director], ['Writer', writer], ['Stars', actor0, actor1]]    
    return credit_list

time: 7.15 ms
